## Laboratorio#6
### Preámbulo

In [2]:
from scapy.all import * # Packet manipulation
import pandas as pd # Pandas - Create and Manipulate DataFrames
import numpy as np # Math Stuff (don't worry only used for one line :] )
import binascii # Binary to Ascii
import seaborn as sns
sns.set(color_codes=True)

Captura de 25 paquetes

In [3]:
num_of_packets_to_sniff = 25
pcap = sniff(count=num_of_packets_to_sniff)

# rdpcap returns packet list
## packetlist object can be enumerated
print(type(pcap))
print(len(pcap))
print(pcap)

<class 'scapy.plist.PacketList'>
25
<Sniffed: TCP:23 UDP:0 ICMP:0 Other:2>


<Ether  dst=40:5b:d8:95:5d:07 src=6c:63:9c:37:ef:60 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=40 id=42706 flags=DF frag=0 ttl=108 proto=tcp chksum=0x663b src=40.126.24.147 dst=192.168.0.9 |<TCP  sport=https dport=39782 seq=1317107098 ack=1437256936 dataofs=5 reserved=0 flags=A window=16385 chksum=0xea40 urgptr=0 |<Padding  load='\x00\x00\x00\x00\x00\x00' |>>>>

In [ ]:
pcap[0]

TCP

In [6]:
"""
0                   1                   2                   3
0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|          Source Port          |       Destination Port        |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|                        Sequence Number                        |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|                    Acknowledgment Number                      |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|  Data |           |U|A|P|R|S|F|                               |
| Offset| Reserved  |R|C|S|S|Y|I|            Window             |
|       |           |G|K|H|T|N|N|                               |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|           Checksum            |         Urgent Pointer        |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|                    Options                    |    Padding    |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|                             data                              |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
02 04 05 a0 01 03 03 05 01 01 08 0a 1d 74 65 c5 00 00 00 00 04 02 00 00
"""
print()

#### Exploración

In [7]:
#Recordemos que en los modelos de red, las capas inferiores encapsulan la información de las capas superiores
#Frames en paquetes, paquetes en segmentos (Ethernet, IP), modelo TCP/IP

ethernet_frame = pcap[0]
ip_packet = ethernet_frame.payload
segment = ip_packet.payload
data = segment.payload #recupera la informacion de la capa 4

print("Informacion del frame:")
print(ethernet_frame.summary())

print("Informacion del paquete:")
print(ip_packet.summary())

print("Informacion del segmento:")
print(segment.summary())

print("Data:")
print(data.summary())

ethernet_frame.show()
data.show()

Informacion del frame:
Ether / IP / TCP 40.126.24.147:https > 192.168.0.9:39782 A / Padding
Informacion del paquete:
IP / TCP 40.126.24.147:https > 192.168.0.9:39782 A / Padding
Informacion del segmento:
TCP 40.126.24.147:https > 192.168.0.9:39782 A / Padding
Data:
Padding
###[ Ethernet ]### 
  dst       = 40:5b:d8:95:5d:07
  src       = 6c:63:9c:37:ef:60
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 40
     id        = 42706
     flags     = DF
     frag      = 0
     ttl       = 108
     proto     = tcp
     chksum    = 0x663b
     src       = 40.126.24.147
     dst       = 192.168.0.9
     \options   \
###[ TCP ]### 
        sport     = https
        dport     = 39782
        seq       = 1317107098
        ack       = 1437256936
        dataofs   = 5
        reserved  = 0
        flags     = A
        window    = 16385
        chksum    = 0xea40
        urgptr    = 0
        options   = ''
###[ Padding ]### 
           l

#### Tipos de datos en Scapy

In [8]:
print(type(ethernet_frame))
print(type(ip_packet))
print(type(segment))

# Packets can be filtered on layers ie) ethernet_frame[scapy.layers.l2.Ether]
ethernet_type = type(ethernet_frame)
ip_type = type(ip_packet)
tcp_type = type(segment)
print("Ethernet",pcap[ethernet_type])
print("IP", pcap[ip_type])
print("TCP", pcap[tcp_type])

# Scapy provides this via import statements
from scapy.layers.l2 import Ether
from scapy.layers.inet import IP
from scapy.layers.inet import TCP, UDP

print("UDP", pcap[UDP])

<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.inet.IP'>
<class 'scapy.layers.inet.TCP'>
Ethernet <Ether from Sniffed: TCP:23 UDP:0 ICMP:0 Other:2>
IP <IP from Sniffed: TCP:23 UDP:0 ICMP:0 Other:1>
TCP <TCP from Sniffed: TCP:23 UDP:0 ICMP:0 Other:0>
UDP <UDP from Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>


#### Conversion de PCAP a DF

In [14]:
# Obtenemos los campos de IP/TCP/UDP que seran las columnas de nuestro DF
ip_fields = [field.name for field in IP().fields_desc]
tcp_fields = [field.name for field in TCP().fields_desc]
udp_fields = [field.name for field in UDP().fields_desc]

#Concatenamos todos los campos
dataframe_fields = ip_fields + ['time'] + tcp_fields + ['payload','payload_raw','payload_hex']

# Creamos un DF en blanco con los campos como columnas
df = pd.DataFrame(columns=dataframe_fields)
for packet in pcap[IP]:
    # Obtenemos los valores de los campos de cada fila
    field_values = []
    # A;adimos todos los valores de los campos de IP al df
    for field in ip_fields:
        if field == 'options':
            # Obtenemos el numero de opciones del IP Header
            field_values.append(len(packet[IP].fields[field]))
        else:
            field_values.append(packet[IP].fields[field])

    field_values.append(packet.time)

    layer_type = type(packet[IP].payload)
    for field in tcp_fields:
        try:
            if field == 'options':
                field_values.append(len(packet[layer_type].fields[field]))
            else:
                field_values.append(packet[layer_type].fields[field])
        except:
            field_values.append(None)

    # Agregamos el payload
    field_values.append(len(packet[layer_type].payload))
    field_values.append(packet[layer_type].payload.original)
    field_values.append(binascii.hexlify(packet[layer_type].payload.original))
    # Agregamos la fila al df
    df_append = pd.DataFrame([field_values], columns=dataframe_fields)
    df = pd.concat([df, df_append], axis=0)

# Reset Index
df = df.reset_index()
# Drop old index column
df = df.drop(columns="index")

df.head(5)

,version,ihl,tos,len,id,flags,frag,ttl,proto,chksum,...,dataofs,reserved,flags,window,chksum,urgptr,options,payload,payload_raw,payload_hex
0,4,5,0,40,42706,DF,0,108,6,26171,...,5,0,A,16385,59968,0,None,6,b'\x00\x00\x00\x00\x00\x00',b'000000000000'
1,4,5,0,1500,42707,DF,0,108,6,24710,...,5,0,A,16385,51347,0,0,1460,b'\x17\x03\x03\x061\xed?wIS\xfa\x14\xba\xf8\xd...,b'1703030631ed3f774953fa14baf8db1c1d3dc18faecf...
2,4,5,0,170,42708,DF,0,108,6,26039,...,5,0,PA,16385,44381,0,0,130,"b""2\x98\xf0't5\xf8\xad\x0e\x9f\xbd\xa7\x11\x8b...",b'3298f0277435f8ad0e9fbda7118b397ae9e89fb6d98d...
3,4,5,0,40,49266,DF,0,64,6,30875,...,5,0,A,1024,8204,0,None,0,b'',b''
4,4,5,0,139,33724,DF,0,108,6,27986,...,5,0,PA,16383,9538,0,0,99,b'\x16\x03\x03\x00X\x02\x00\x00T\x03\x03\xcf!\...,b'1603030058020000540303cf21ad74e59a6111be1d8c...


In [11]:
# Return Src Address, Dst Address, Src Port, Dst Port
df[['src','dst','sport','dport']]

,src,dst,sport,dport
0,40.126.24.147,192.168.0.9,443,39782
1,40.126.24.147,192.168.0.9,443,39782
2,40.126.24.147,192.168.0.9,443,39782
3,192.168.0.9,40.126.24.147,39782,443
4,20.42.72.131,192.168.0.9,443,39783
5,192.168.0.9,20.42.72.131,39783,443
6,192.168.0.9,40.126.24.147,39782,443
7,20.42.72.131,192.168.0.9,443,39783
8,20.42.72.131,192.168.0.9,443,39783
9,192.168.0.9,20.42.72.131,39783,443


In [15]:
#Calcular estadisticas
print("# Top Source Address")
df['src'].describe()

# Top Source Address


count              24
unique              3
top       192.168.0.9
freq               12
Name: src, dtype: object

In [24]:
#¿Quien es la IP de origen mas frecuente?
frequent_address = df['src'].describe()['top']
print("Dirección más frecuente:",frequent_address)

# A que IPs les habla la IP mas frecuente
unique_dst = df[df['src'] == frequent_address]['dst'].unique()
print("\nIPs destino más frecuentes")
for address in unique_dst:
    print("->",address)

Dirección más frecuente: 192.168.0.9

IPs destino más frecuentes
-> 40.126.24.147
-> 20.42.72.131
-> 224.0.0.22


### Análisis estadístico